# F# によるデータ解析・機械学習のデモ

この notebook は kaggle のチュートリアルコンペティションである [Titanic](https://www.kaggle.com/competitions/titanic) のデータを F# (w/ Jupyter Notebook)を用いて解析するデモンストレーションです．

以下の構成でファイルが格納されていることを想定しています：

- data/ : データファイル．[kaggle からダウンロードする](https://www.kaggle.com/competitions/titanic)．
  - train.csv : 学習用データ．
  - test.csv : テスト用データ
- src/ : スクリプト．
  - main.ipynb : このファイル．
  - Utils.fsx : ユーティリティ関数を定義したファイル．
- submission/ : 提出用ファイル．


依存パッケージを読み込みます．


In [21]:
#r "nuget: Deedle"
#r "nuget: Plotly.NET.Interactive"
#r "nuget: Accord.MachineLearning"
#r "nuget: Accord.Statistics"

Installed Packages Accord.MachineLearning, 3.8.0 Accord.Statistics, 3.8.0 Deedle, 3.0.0 Plotly.NET.Interactive, 4.2.1

Loading extensions from `C:\Users\User\.nuget\packages\plotly.net.interactive\4.2.1\interactive-extensions\dotnet\Plotly.NET.Interactive.dll`

## Deedle ... データフレームライブラリ

最初に，データフレームライブラリ [Deedle](https://bluemountaincapital.github.io/Deedle/) を用いてタイタニック号の乗客データを読み込みます．

### データの読み込み

`Deedle.Frame` はデータフレームを格納する型です．配置したトレーニングファイル，テストファイルを `Frame.ReadCsv` 関数で読み込み，`Frame.indexRowsInt` 関数で `PassengerId` を行インデックスに設定します．


In [22]:
open Deedle

let train =
    Frame.ReadCsv(__SOURCE_DIRECTORY__ + "/../data/titanic/train.csv") // トレーニングデータを読み込み，
    |> Frame.indexRowsInt "PassengerId" // PassengerId を行名に設定する．

let test =
    Frame.ReadCsv(__SOURCE_DIRECTORY__ + "/../data/titanic/test.csv") // テストデータを読み込み，
    |> Frame.indexRowsInt "PassengerId" // PassengerId を行名に設定する．

// テストデータの正答率をローカルで調べたい場合は代わりに以下のコードで `train` を分割して読み込む．

// let titanic =
//     Frame.ReadCsv(__SOURCE_DIRECTORY__ + "/../data/titanic/train.csv") |> Frame.indexRowsInt "PassengerId"

// let train = titanic.GetRowsAt([|0..790|])
// let test = titanic.GetRowsAt([|791..890|])

In [23]:
// トレーニングデータを表示する．
train.Print()

       Survived Pclass Name                                                Sex    Age       SibSp Parch Ticket           Fare    Cabin Embarked 
1   -> False    3      Braund, Mr. Owen Harris                             male   22        1     0     A/5 21171        7.25          S        
2   -> True     1      Cumings, Mrs. John Bradley (Florence Briggs Thayer) female 38        1     0     PC 17599         71.2833 C85   C        
3   -> True     3      Heikkinen, Miss. Laina                              female 26        0     0     STON/O2. 3101282 7.925         S        
4   -> True     1      Futrelle, Mrs. Jacques Heath (Lily May Peel)        female 35        1     0     113803           53.1    C123  S        
5   -> False    3      Allen, Mr. William Henry                            male   35        0     0     373450           8.05          S        
6   -> False    3      Moran, Mr. James                                    male   <missing> 0     0     330877           8.4583   

In [24]:
// テストデータを表示する．
test.Print()

        Pclass Name                                                    Sex    Age       SibSp Parch Ticket             Fare    Cabin Embarked 
892  -> 3      Kelly, Mr. James                                        male   34.5      0     0     330911             7.8292        Q        
893  -> 3      Wilkes, Mrs. James (Ellen Needs)                        female 47        1     0     363272             7             S        
894  -> 2      Myles, Mr. Thomas Francis                               male   62        0     0     240276             9.6875        Q        
895  -> 3      Wirz, Mr. Albert                                        male   27        0     0     315154             8.6625        S        
896  -> 3      Hirvonen, Mrs. Alexander (Helga E Lindqvist)            female 22        1     1     3101298            12.2875       S        
897  -> 3      Svensson, Mr. Johan Cervin                              male   14        0     0     7538               9.225         S        

今回のデータフレームは `Frame<int, string>` という型を持ちます．これは行のインデックスが `int` 型，列のインデックスが `string` 型であることを意味します．


### 可視化

Plotly.NET ライブラリを用いてデータを可視化します．
ノートブックで用いる場合には `Plotly.NET.Interactive` を読み込んで使用します．


In [32]:
open Plotly.NET

let x: float seq = train.GetColumn "Age" |> Series.values

Chart.Histogram(X = x)


<!-- Plotly chart will be drawn inside this DIV -->

### データクレンジング・データの前処理

上で表示した通り，データフレームのいくつかの列には欠損値が含まれています．データ解析の前にこれらを埋める必要があります．また，整数や boolean，文字列であるような列の値を変換し，浮動小数点型のみを含むデータフレームに変換します．

Deedle のデータフレームでは，欠損したセルの値をピンポイントで読み込もうとしたタイミングで例外が発生します．また，`Series` を読み込む関数，たとえば `Series.values` では欠損値はスキップされ，値の個数が減少した列が返される点に注意が必要です．欠損値を `None`，欠損値でない値を `Some` として取り出すには，たとえば `Series.valuesAll` のように末尾に `All` を付けた関数を用います．

まずは欠損値の数を表示する関数 `printCleansingInfo` を作成しましょう．なお，データ上は `Embarked` や `Cabin` なども欠損値を含みますが，文字列型の列については欠損値は空文字列として取り込まれているので，これらは欠損値として扱わないことにします．


In [14]:
let printCleansingInfo (frame: Frame<'R, 'C>) =
    let missingValueCounts =
        frame.Columns
        |> Series.observations
        |> Seq.map (
            fun (key, col) ->
                let length = col.ValuesAll |> Seq.length // 欠損値も含めた列の長さ．
                let valueCount = col.ValueCount // 欠損値を除いた列の長さ．
                let missingCount = length - valueCount // 欠損値の数．
                key, missingCount
        )
        |> Seq.filter (fun (key, missingCount) -> missingCount > 0)
    
    missingValueCounts
    |> Seq.iter (fun (key, count) -> printfn "Column %A has %d missing values" key count)

    if missingValueCounts |> Seq.isEmpty then
        printfn "No missing values"

printfn "Train data:"
printCleansingInfo train

printfn "Test data:"
printCleansingInfo test

Train data:
Column "Age" has 177 missing values
Test data:
Column "Age" has 86 missing values
Column "Fare" has 1 missing values


これらの欠損値がなくなり，すべてのセルの値が `float` 型になるよう，データフレームを変換します．
以下では `Utils.fsx` 内で定義した関数を用いて各列を変換していきます．


In [15]:
// one-hot エンコーディング．
let oneHotEncode (colName: string) (categories: string list) (frame: Frame<'R, string>) =
    // 元の列を取得する．
    let originalCol = frame.GetColumn colName

    // カテゴリを表す列を追加する．
    let addCategoryCol (category: string) (frame: Frame<'R, string>) =
        let col =
            originalCol
            |> Series.mapValues (fun v -> if v = category then 1.0 else 0.0)
        
        frame
        |> Frame.addCol (colName + "_" + category) col

    frame
    // 元の列を削除し，
    |> Frame.dropCol colName
    // categories の各要素 に対して，addCategoryCol を適用する．
    |> Seq.foldBack addCategoryCol categories

let preprocess (frame: Frame<int, string>) =
    frame
    // Name, Ticket, Cabin は削除する．
    |> Frame.dropCol "Name"
    |> Frame.dropCol "Ticket"
    |> Frame.dropCol "Cabin"
    // Sex は `male` ならば `0.0`，`female` ならば `1.0` に置き換える．
    |> Frame.replaceCol "Sex" (frame.GetColumn "Sex" |> Series.mapValues (fun s -> if s = "male" then 0 else 1))
    // Embarked は one-hot エンコーディングを行う．
    |> oneHotEncode "Embarked" [ "S"; "Q"; "C" ]
    // Fare は欠損値を平均値で置き換える．
    |> Frame.replaceCol "Fare" (
        frame.GetColumn "Fare"
        |> Series.fillMissingWith (frame?Fare |> Stats.mean)
    )
    // Age は欠損値を平均値で置き換え，そのうえで欠損値であるか示す列を追加する．
    |> Frame.replaceCol "Age" (
        frame.GetColumn "Age"
        |> Series.fillMissingWith (frame?Age |> Stats.mean)
    )
    |> Frame.addCol "Age_missing" (
        frame.GetColumn "Age"
        |> Series.mapValues (fun _ -> 0)
        |> Series.fillMissingWith 1
    )

let train' =
    train
    |> Frame.replaceCol "Survived" (train?Survived |> Series.mapValues float)
    |> preprocess
let test' = test |> preprocess

// test が Survived を含むなら以下のコードで Survived を変換する．
// let test' = test |> preprocess |> Frame.replaceCol "Survived" (test?Survived |> Series.mapValues float)

train'.Print()
test'.Print()

       Pclass SibSp Parch Survived Sex Embarked_C Embarked_Q Embarked_S Fare    Age               Age_missing 
1   -> 3      1     0     0        0   0          0          1          7.25    22                0           
2   -> 1      1     0     1        1   1          0          0          71.2833 38                0           
3   -> 3      0     0     1        1   0          0          1          7.925   26                0           
4   -> 1      1     0     1        1   0          0          1          53.1    35                0           
5   -> 3      0     0     0        0   0          0          1          8.05    35                0           
6   -> 3      0     0     0        0   0          1          0          8.4583  29.69911764705882 1           
7   -> 1      0     0     0        0   0          0          1          51.8625 54                0           
8   -> 3      3     1     0        0   0          0          1          21.075  2                 0           
9

In [16]:
// 欠損値を改めて確認する．
printfn "Train data:"
printCleansingInfo train'
printfn "Test data:"
printCleansingInfo test'

Train data:
No missing values
Test data:
No missing values


## Accord.NET ... 機械学習ライブラリ

続いて [Accord.NET](http://accord-framework.net/) を用いてデータの学習を行います．今回は多変量ロジスティック回帰分析とランダムフォレストをデモンストレートします．

まずはデータフレームから予測変数と目的変数の配列を取り出します．


In [17]:
let inputs: float array array =
    train'
    |> Frame.dropCol "Survived"
    |> Frame.toJaggedArray

let survived: int array =
    train'
    |> Frame.getCol "Survived"
    |> Series.values
    |> Seq.toArray

次に，ロジスティック回帰とランダムフォレストのモデルを作成し，学習を行います．

それぞれのモデルは `Accord.Statistics.Analysis.MultivariateLogisticRegression` および `Accord.MachineLearning.DecisionTrees.Learning.C45Learning` に存在します(ランダムフォレストのアルゴリズムはいくつか選択することができます)．モデルを作成し，`.Learn(inputs, survived)` でトレーニングデータを学習させます．


In [18]:
open Accord
open Accord.Statistics.Analysis
open Accord.MachineLearning.DecisionTrees.Learning

// Multinomial logistic regression.
let MLR = MultinomialLogisticRegressionAnalysis().Learn(inputs, survived)

// Random forest.
let RF = C45Learning().Learn(inputs, survived)

// 入力から生存者を予測する関数を取得する．
let MLRDecide: float array -> int = MLR.Decide
let RFDecide: float array -> int = RF.Decide

予測関数 `decide: float array -> int` を用いてテストデータの生存者を予測する関数を定義します．


In [19]:
// テストデータの予測結果を計算する．
let predictions (decide: float array -> int) =
    test'
    |> Frame.rows
    |> Series.observations
    |> Seq.map (fun (id, row) ->
        id,
        row.Values
        |> Seq.map Convert.ToDouble
        |> Seq.toArray
        |> decide
    )
    |> Seq.map (fun (id, pred) -> {|
        PassengerId = id
        Survived = pred
    |})

テストデータが `Survived` を含むなら，以下の関数で正答率を計算できます．


In [20]:
// 正答率を計算する．
let accuracy (predictions: {| PassengerId: int; Survived: int |} seq) =
    let actual =
        test'
        |> Frame.getCol "Survived"
        |> Series.values
        |> Seq.map (int: float -> int)

    let correctCount =
        predictions
        |> Seq.map (fun p -> p.Survived)
        |> Seq.zip actual
        |> Seq.filter (fun (pred, act) -> pred = act)
        |> Seq.length
    
    float correctCount / float (Seq.length actual)

kaggle への提出用の csv ファイルを作成する関数を定義します．


In [21]:
// 予測結果を出力する．
let export (fileName: string) (predictions: {| PassengerId: int; Survived: int |} seq) =
    let header = "PassengerId,Survived\n"
    let body =
        predictions
        |> Seq.map (fun p -> sprintf "%d,%d" p.PassengerId p.Survived)
        |> String.concat "\n"
    
    let out = header + body
    File.WriteAllText (__SOURCE_DIRECTORY__ + "/../submission/titanic/" + fileName + ".csv", out)

以上の関数を，ロジスティック回帰とランダムフォレストのそれぞれについて実行します．


In [ ]:
[
    "MLR", MLRDecide
    "RF", RFDecide
]
|> Seq.iter (fun (name, decide) ->
    printfn "Decider: %s" name

    let predictions = predictions decide

    predictions
    |> Seq.take 10
    |> Seq.iter (fun p -> printfn "PassengerId: %d, Survived: %d" p.PassengerId p.Survived)

    printfn "..."

    if test'.ColumnKeys |> Seq.contains "Survived" then
        predictions |> accuracy |> printfn "Accuracy: %f"
    
    predictions |> export (name + "-submission")
)

今回は特徴量エンジニアリングなどを特に行っていませんが，実践的にはデータの前処理の際にグルーピングなどの処理を挿入してさらに精度を向上させることもできます．
